In [8]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os

sz = 128
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
#classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
#classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=19, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


# Step 2 - Preparing the train/test data and training the model
classifier.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 28800)             0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [9]:
# Code copied from - https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
training_set = train_datagen.flow_from_directory('../Local/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('../Local/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 8865 images belonging to 19 classes.
Found 2998 images belonging to 19 classes.


In [13]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=887, # No of images in training set
        epochs=5,
        validation_data=test_set,
        validation_steps=214)# No of images in test set

C:\Users\shubh\AppData\Local\Temp/ipykernel_5244/801791507.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/5
887/887 [==============================] - 81s 91ms/step - loss: 0.7054 - accuracy: 0.7474 - val_loss: 0.0894 - val_accuracy: 0.9743
Epoch 2/5
887/887 [==============================] - 82s 92ms/step - loss: 0.5423 - accuracy: 0.8095 - val_loss: 0.0631 - val_accuracy: 0.9771
Epoch 3/5
887/887 [==============================] - 81s 91ms/step - loss: 0.4540 - accuracy: 0.8430 - val_loss: 0.0600 - val_accuracy: 0.9808
Epoch 4/5
887/887 [==============================] - 81s 92ms/step - loss: 0.3478 - accuracy: 0.8788 - val_loss: 0.0433 - val_accuracy: 0.9808
Epoch 5/5
887/887 [==============================] - 112s 126ms/step - loss: 0.3062 - accuracy: 0.8943 - val_loss: 0.0222 - val_accuracy: 0.9953


In [49]:
import numpy as np
from keras.preprocessing import image

test_file = './Test/A/108.jpg'


In [50]:
import matplotlib.pyplot as plt
import cv2

In [51]:
test_img = image.load_img(test_file,target_size=(128, 128))

In [52]:
test_img = image.img_to_array(test_img)

In [53]:
test_img = cv2.cvtColor(np.array(test_img),cv2.COLOR_RGB2GRAY)

In [57]:
test_img.shape

(128, 128)

In [58]:
test_img = np.expand_dims(test_img, axis=0)
test_img = test_img/255

In [59]:
prediction_prob = classifier.predict(test_img)

In [60]:
prediction_prob.c

array([[9.9847120e-01, 9.9793262e-08, 7.3163368e-12, 1.5505442e-11,
        4.1627394e-16, 1.2632082e-13, 2.9920094e-11, 3.6385703e-16,
        2.5187287e-06, 2.3772621e-11, 2.3731323e-13, 2.5238109e-14,
        1.5261973e-03, 7.5182629e-12, 5.4616884e-14, 5.2393240e-15,
        8.7491958e-12, 1.1798728e-08, 1.0830084e-14]], dtype=float32)

In [61]:
prediction_prob.shape

(1, 19)

In [66]:
max_val = prediction_prob[0][0]
for i in range(len(prediction_prob[0])):
    if prediction_prob[0][i]<max_val:
        prediction_prob[0][i]=0
    else:
        prediction_prob[0][i]=1

In [67]:
prediction_prob

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype=float32)

In [68]:
training_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'G': 3,
 'I': 4,
 'J': 5,
 'K': 6,
 'L': 7,
 'M': 8,
 'N': 9,
 'Q': 10,
 'R': 11,
 'S': 12,
 'T': 13,
 'V': 14,
 'W': 15,
 'X': 16,
 'Y': 17,
 'Z': 18}